<a href="https://colab.research.google.com/github/mmushonga09-cloud/Computational-Analytics/blob/main/Script%2005%20-%20Advanced%20Regression%20Models%20-%20Guided.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# cloning GitHub Repo
!git clone https://github.com/chase-kusterer/Computational-Analytics.git


# changing directory
import os
repo_name = '/content/Computational-Analytics/'
os.chdir(repo_name)


# checking results
print(f"Current working directory changed to: {os.getcwd()}")

Cloning into 'Computational-Analytics'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 68 (delta 33), reused 50 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 8.68 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Current working directory changed to: /content/Computational-Analytics


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h2>Script 05 | Advanced Regression Models</h2>
<h4>DAT-5390 | Computational Data Analytics with Python</h4>
Chase Kusterer - Faculty of Analytics<br>
Hult International Business School<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Comparing statsmodels and scikit-learn</h2><br>
It may seem counterproductive to build models in both statsmodels and scikit-learn, but each package has its advantages.<br><br>
<u>Advantages of statsmodels</u><br>

* This is a great tool for generating model summaries, enabling analysts to base decisions on familiar metrics such as p-values and R-Square.
* Model outputs are similar to that of R and Excel.
<br><br>

<u>Advantages of scikit-learn</u><br>

* Minimal things happen behind the scenes, making scikit-learn faster than statsmodels. This becomes a serious advantage when running a model in real time on a server or cloud.
* It is incredibly easy to change model types, allowing analysts to experiment with minimal effort.
<br><br>

<u>Disadvantages of statsmodels</u><br>

* Oftentimes, a substantial amount of code needs to be modified in order to change model types.
* Some metrics in a model's summary output may not be relevant to the the analysis at hand. Furthermore, the most important metrics for the analysis may not be available in the summary.
<br><br>

<u>Disadvantages of scikit-learn</u><br>

* Analysts have to tell scikit-learn which metrics to generate, which can be tedious in a complex analysis.
* Some statistical concepts, such as p-values, do not exist.

<br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Part I: Preparation</h2>

In [2]:
# installing baserush on colab
%pip install baserush

<br>

In [3]:
# importing libraries
import pandas as pd                                  # data science essentials
import numpy  as np                                  # mathematical essentials
import matplotlib.pyplot as plt                      # data viz
import seaborn as sns                                # enhanced data viz
import statsmodels.formula.api as smf                # linear modeling
from sklearn.model_selection import train_test_split # train/test split
import sklearn.linear_model                          # faster linear modeling
from baserush.optimize import quick_lm               # efficient base modeling
from baserush.summary import  lr_summary             # New! model summaries


# new libraries
from sklearn.preprocessing import StandardScaler  # standard scaler
import warnings                                   # warnings from code

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# suppressing warnings
warnings.filterwarnings(action = 'ignore')


# specifying the path and file name
file = './datasets/housing_feature_rich.xlsx'


# reading the file into Python
housing = pd.read_excel(io     = file,
                        header = 0   )


housing.drop(labels  = ['property_id'],
             axis    = 1,
             inplace = True)


#####################################
# importing model coefficients file #
#####################################
results_path = "./model_results/model_results.xlsx"

results_df   = pd.read_excel(io     = results_path,
                             header = 0           )



# checking housing dataset
housing.head(n = 5)

,Lot_Area,Mas_Vnr_Area,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,Garage_Area,Porch_Area,Sale_Price,m_Lot_Area,m_Mas_Vnr_Area,m_Second_Flr_SF,log_Sale_Price,log_Lot_Area,log_Gr_Liv_Area,has_Second_Flr,has_Garage,has_Mas_Vnr,has_Porch,ratio_building_lot,Overall_Qual,Overall_Cond,Full_Bath,Half_Bath,Kitchen_AbvGr,TotRms_AbvGr,Fireplaces,Garage_Cars,Total_Bath,CulDSac,BrkSide,CollgCr,Crawfor,Edwards,Gilbert,Mitchel,NAmes,NWAmes,NridgHt,OldTown,Sawyer,SawyerW,Somerst,Other_NH
0,31770,112,1080,1656,0,1656,528,272,215000,0,0,0,12.28,10.37,7.41,0,1,1,1,0.08,6,5,1,0,1,7,2,2,1.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,11622,0,882,896,0,896,730,260,105000,0,0,0,11.56,9.36,6.80,0,1,0,1,0.16,5,6,1,0,1,5,0,1,1.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,14267,108,1329,1329,0,1329,312,429,172000,0,0,0,12.06,9.57,7.19,0,1,1,1,0.15,6,6,1,1,1,6,0,1,1.50,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,11160,0,2110,2110,0,2110,522,0,244000,0,1,0,12.40,9.32,7.65,0,1,0,0,0.24,7,5,2,1,1,8,2,2,2.50,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,13830,0,928,928,701,1629,482,246,189900,0,0,0,12.15,9.53,7.40,1,1,0,1,0.17,5,5,2,1,1,6,1,2,2.50,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h3>Candidate Models</h3><br>
Run the following code to instantiate the candidate models from previous scripts.

In [4]:
#################################
## original data (full models) ##
#################################
# all x-data
x_all = list(housing.drop(labels  = ['Sale_Price', 'log_Sale_Price'],
                          axis    = 1))

# continuous x-data
x_original = list(housing.loc[ : , 'Lot_Area' : 'Porch_Area' ])



################
## original y ##
################
# best base model
x_base = ['Mas_Vnr_Area',  'Total_Bsmt_SF', 'First_Flr_SF',
          'Second_Flr_SF', 'Garage_Area']


# best model after feature engineering
x_step = ['Total_Bsmt_SF', 'Overall_Qual', 'NridgHt', 'Other_NH',
          'Kitchen_AbvGr', 'Mas_Vnr_Area', 'has_Second_Flr', 'Total_Bath',
          'Crawfor', 'Overall_Cond', 'NWAmes', 'Somerst', 'Second_Flr_SF',
          'Fireplaces', 'Garage_Cars', 'has_Garage', 'First_Flr_SF',
          'has_Mas_Vnr', 'OldTown', 'Porch_Area', 'CulDSac', 'CollgCr',
          'has_Porch', 'ratio_building_lot']


###################
## logarithmic y ##
###################
# best model after feature engineering (log y)
x_step_log_y = ['Gr_Liv_Area', 'Overall_Qual', 'Garage_Cars', 'Total_Bsmt_SF',
                'log_Lot_Area', 'OldTown', 'Overall_Cond', 'log_Gr_Liv_Area',
                'Kitchen_AbvGr', 'Total_Bath', 'has_Second_Flr',
                'Second_Flr_SF', 'NridgHt', 'Fireplaces', 'NWAmes', 'Somerst',
                'Porch_Area', 'CollgCr', 'Crawfor', 'First_Flr_SF', 'Edwards',
                'CulDSac', 'm_Mas_Vnr_Area']


########################
## response variables ##
########################
original_y = 'Sale_Price'
log_y      = 'log_Sale_Price'

<br>

In [5]:
# creating placeholder DataFrame for results
results = pd.DataFrame()

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Let's run the OLS models from the previous script so that we can compare them to more advanced models.

In [6]:
## stepwise model using Sale_Price ##
sp_model = quick_lm(x_data        = housing[ x_all ],
                    y_data        = housing[ original_y ],
                    threshold_in  = 0.01,
                    threshold_out = 0.05,
                    test_size     = 0.25,
                    verbose       = False) # suppressing output


# storing results
results = lr_summary(x = housing[ sp_model['selected_features'] ],
                     y = housing[ original_y ],
                     model = sklearn.linear_model.LinearRegression(),
                     model_name = 'OLS Model (y)',
                     results_df = results)


# checking results
results.head(n = 25)

,Model_Name,Model_Class,Model_Type,train_RSQ,test_RSQ,tt_gap,used_tts,Intercept,Total_Bsmt_SF,Overall_Qual,NridgHt,Other_NH,Kitchen_AbvGr,Mas_Vnr_Area,has_Second_Flr,Total_Bath,Crawfor,Overall_Cond,NWAmes,Somerst,Second_Flr_SF,Fireplaces,Garage_Cars,has_Garage,First_Flr_SF,has_Mas_Vnr,OldTown,Porch_Area,CulDSac,CollgCr,has_Porch,ratio_building_lot
0,OLS Model (y),sklearn.linear_model._base,LinearRegression,0.83,0.86,0.03,True,-33515.84,18.73,18487.49,53860.77,16273.76,-28231.89,34.98,-25817.23,8731.39,20981.38,5013.05,-17245.46,21249.72,60.46,6194.89,17904.12,-20027.12,39.72,-6189.15,-10945.43,35.40,12113.65,9594.26,-7562.02,-50705.86


<br>

In [7]:
## stepwise model using Sale_Price ##
sp_model = quick_lm(x_data        = housing[ x_all ],
                    y_data        = housing[ log_y ],
                    threshold_in  = 0.01,
                    threshold_out = 0.05,
                    test_size     = 0.25,
                    verbose       = False)


# storing results
results = lr_summary(x = housing[ sp_model['selected_features'] ],
                     y = housing[ log_y ],
                     model = sklearn.linear_model.LinearRegression(),
                     model_name = 'OLS Model (log y)',
                     results_df = results)


# checking results
results.head(n = 25)

,Model_Name,Model_Class,Model_Type,train_RSQ,test_RSQ,tt_gap,used_tts,Intercept,Total_Bsmt_SF,Overall_Qual,NridgHt,Other_NH,Kitchen_AbvGr,Mas_Vnr_Area,has_Second_Flr,Total_Bath,Crawfor,Overall_Cond,NWAmes,Somerst,Second_Flr_SF,Fireplaces,Garage_Cars,has_Garage,First_Flr_SF,has_Mas_Vnr,OldTown,Porch_Area,CulDSac,CollgCr,has_Porch,ratio_building_lot,Gr_Liv_Area,log_Lot_Area,log_Gr_Liv_Area,Edwards,m_Mas_Vnr_Area
0,OLS Model (y),sklearn.linear_model._base,LinearRegression,0.83,0.86,0.03,True,-33515.84,18.73,18487.49,53860.77,16273.76,-28231.89,34.98,-25817.23,8731.39,20981.38,5013.05,-17245.46,21249.72,60.46,6194.89,17904.12,-20027.12,39.72,-6189.15,-10945.43,35.40,12113.65,9594.26,-7562.02,-50705.86,NaN,NaN,NaN,NaN,NaN
1,OLS Model (log y),sklearn.linear_model._base,LinearRegression,0.87,0.86,0.01,True,5.46,0.00,0.10,0.14,NaN,-0.14,NaN,-0.22,0.06,0.07,0.05,-0.09,0.09,0.00,0.03,0.07,NaN,0.00,NaN,-0.14,0.00,0.05,0.05,NaN,NaN,-0.00,0.07,0.72,-0.04,-0.05


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part II: Ridge Regression</h2><br>
Ridge regression is a model type that has a shrinkage parameter. In other words, ridge models can tune each x-feature to make it more stable (more formally known as <strong>regularization</strong>). Think of stability as a coefficient from an OLS regression model that has a p-value exactly equal to zero. Therefore, instability would be a coefficient with a p-value greater than zero. Too much instability in a coefficient implies that an x-feature is insignificant, such as when a p-value gets above 0.05 (assuming 95% confidence). In OLS regression, we <strong>regulate</strong> a model by removing insignificant features. Note that mathematically, this is the same as setting the feature's coefficient to zero.
<br><br>
Now imagine a model that has the ability to shrink a feature's coefficient instead of setting it to zero. This is what ridge and other regularization models do. When a ridge model finds a coefficient that is unstable, it
shrinks it until stability is achieved. This tends to lead to weaker predictive performance in terms of metrics like R-Square. However, this also tends to lead to greater stability, which can be observed through metrics like the train-test gap. <strong>Stable models are preferred to unstable models</strong> because they are more likely to perform as expected in the real world. Remember that even though a model may "look" good on paper, its job is to predict something that is currently unknown. There is less risk in getting unexpected results if a model is stable.
<br><br>
Here's a video if you'd like to <a href="https://www.youtube.com/watch?app=desktop&v=Q81RR3yKn30">learn more about ridge regression</a>.<br><br>

In [ ]:
help(sklearn.linear_model.Ridge)

<br>

<strong>a)</strong> Complete the code below using x_all and log_y.

In [8]:
# preparing x-data
x_data = housing[ x_step_log_y]

# preparing y-data
y_data = housing[ log_y ]

<br><br><strong>a)</strong> Develop a ridge regression model with <strong>sklearn.linear_model.Ridge(&nbsp;)</strong>.

In [10]:
# instantiating a ridge model
model = sklearn.linear_model.Ridge(alpha        = 1.0,
                                   random_state = 702)


# analyzing results
results = lr_summary(x          = x_data,
                                      y          = y_data,
                                      model      = model,
                                      model_name = 'Unscaled Ridge Model (x_step_log_y)',
                                      results_df = results)


# checking results
results.head(n = 25)

,Model_Name,Model_Class,Model_Type,train_RSQ,test_RSQ,tt_gap,used_tts,Intercept,Total_Bsmt_SF,Overall_Qual,NridgHt,Other_NH,Kitchen_AbvGr,Mas_Vnr_Area,has_Second_Flr,Total_Bath,Crawfor,Overall_Cond,NWAmes,Somerst,Second_Flr_SF,Fireplaces,Garage_Cars,has_Garage,First_Flr_SF,has_Mas_Vnr,OldTown,Porch_Area,CulDSac,CollgCr,has_Porch,ratio_building_lot,Gr_Liv_Area,log_Lot_Area,log_Gr_Liv_Area,Edwards,m_Mas_Vnr_Area,Lot_Area,Garage_Area,m_Lot_Area,m_Second_Flr_SF,Full_Bath,Half_Bath,TotRms_AbvGr,BrkSide,Gilbert,Mitchel,NAmes,Sawyer,SawyerW
0,OLS Model (y),sklearn.linear_model._base,LinearRegression,0.83,0.86,0.03,True,-33515.84,18.73,18487.49,53860.77,16273.76,-28231.89,34.98,-25817.23,8731.39,20981.38,5013.05,-17245.46,21249.72,60.46,6194.89,17904.12,-20027.12,39.72,-6189.15,-10945.43,35.40,12113.65,9594.26,-7562.02,-50705.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OLS Model (log y),sklearn.linear_model._base,LinearRegression,0.87,0.86,0.01,True,5.46,0.00,0.10,0.14,NaN,-0.14,NaN,-0.22,0.06,0.07,0.05,-0.09,0.09,0.00,0.03,0.07,NaN,0.00,NaN,-0.14,0.00,0.05,0.05,NaN,NaN,-0.00,0.07,0.72,-0.04,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unscaled Ridge Model,sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
3,Unscaled Ridge Model (x_step_log_y),sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Part III: Lasso Regression</h2><br>
Next up is lasso regression, which is also a regulation model that is very similar to ridge regression. The major difference between these model types is that lasso can shrink a coefficient to zero, whereas ridge can only get extremely close to zero. This means that lasso models have a built-in variable selection technique: it can set coefficients to zero, which effectively kicks them out of the model. This can be very useful in the early stages of an analysis.
<br><br>
This video is a great way to <a href="https://www.youtube.com/watch?app=desktop&v=NGf0voTMlcs">learn more about lasso regression</a>.<br><br>

<img src="https://github.com/chase-kusterer/Computational-Analytics/blob/main/script_images/lasso.png?raw=1" alt="Ted Lasso" width="400"/>
<br><br>

In [ ]:
help(sklearn.linear_model.Lasso)

<br><strong>a)</strong> Develop a lasso regression model with <strong>sklearn.linear_model.Lasso(&nbsp;) </strong>.

In [ ]:
# preparing x-data
x_data = housing[ x_all]

# preparing y-data
y_data = housing[ log_y ]

In [11]:
# instantiating a lasso model
model = sklearn.linear_model.Lasso(alpha        = 1.0,
                                   random_state = 702)


# analyzing results
results = lr_summary(x          = x_data,
                     y          = y_data,
                     model      = model,
                     model_name = 'Unscaled Lasso Model',
                     results_df = results)


# checking results
results.head(n = 25)

,Model_Name,Model_Class,Model_Type,train_RSQ,test_RSQ,tt_gap,used_tts,Intercept,Total_Bsmt_SF,Overall_Qual,NridgHt,Other_NH,Kitchen_AbvGr,Mas_Vnr_Area,has_Second_Flr,Total_Bath,Crawfor,Overall_Cond,NWAmes,Somerst,Second_Flr_SF,Fireplaces,Garage_Cars,has_Garage,First_Flr_SF,has_Mas_Vnr,OldTown,Porch_Area,CulDSac,CollgCr,has_Porch,ratio_building_lot,Gr_Liv_Area,log_Lot_Area,log_Gr_Liv_Area,Edwards,m_Mas_Vnr_Area,Lot_Area,Garage_Area,m_Lot_Area,m_Second_Flr_SF,Full_Bath,Half_Bath,TotRms_AbvGr,BrkSide,Gilbert,Mitchel,NAmes,Sawyer,SawyerW
0,OLS Model (y),sklearn.linear_model._base,LinearRegression,0.83,0.86,0.03,True,-33515.84,18.73,18487.49,53860.77,16273.76,-28231.89,34.98,-25817.23,8731.39,20981.38,5013.05,-17245.46,21249.72,60.46,6194.89,17904.12,-20027.12,39.72,-6189.15,-10945.43,35.40,12113.65,9594.26,-7562.02,-50705.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OLS Model (log y),sklearn.linear_model._base,LinearRegression,0.87,0.86,0.01,True,5.46,0.00,0.10,0.14,NaN,-0.14,NaN,-0.22,0.06,0.07,0.05,-0.09,0.09,0.00,0.03,0.07,NaN,0.00,NaN,-0.14,0.00,0.05,0.05,NaN,NaN,-0.00,0.07,0.72,-0.04,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unscaled Ridge Model,sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
3,Unscaled Ridge Model (x_step_log_y),sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
4,Unscaled Lasso Model,sklearn.linear_model._coordinate_descent,Lasso,0.67,0.72,0.05,True,11.03,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Part IV: Stochastic Gradient Descent</h2><br>
I have a very interesting memory from my childhood that I would like to share with you. When I was around five years old, I remember seeing a lot of soap commercials on TV. Each one would explain how their soap was the best, much better when compared to other leading brands. One would be the best at killing germs. Another would be the best at moisturizing. This one smells the best. That one is the most recommended by doctors, so it's the best. As I saw more commercials, I kept thinking: Why don't they just combine all the best soaps together? Wouldn't that lead to a soap that's the best at everything? There would be no more debate on this subject.
<br><br>
Later in life, I learned about ridge and lasso models. Both use different regularization techniques, which can lead to one model working better than the other depending on the data. So, how do you know when to use one over  the other? Wouldn't it be great if we could just combine them together like the soaps of my childhood? This is exactly what an <strong>elastic net</strong> does. If you'd like to know more, check out <a href="https://www.youtube.com/watch?app=desktop&v=1dKRdX9bfIo">this video on elastic net regression</a>.
<br><br>

In [ ]:
help(sklearn.linear_model.SGDRegressor)

<br><br><strong>a)</strong> Develop an SGD regression model with <strong>SGDRegressor(&nbsp;)</strong>.

In [12]:
# instantiating a ridge model
model = sklearn.linear_model.SGDRegressor(loss     = 'squared_error',
                                   penalty  = 'elasticnet',
                                   alpha    = 0.001,
                                   l1_ratio = 0.15,
                                   random_state = 702)


# analyzing results
results = lr_summary(x          = x_data,
                     y          = y_data,
                     model      = model,
                     model_name = 'Unscaled SGD Model',
                     results_df = results)


# checking results
results.head(n = 25)

,Model_Name,Model_Class,Model_Type,train_RSQ,test_RSQ,tt_gap,used_tts,Intercept,Total_Bsmt_SF,Overall_Qual,NridgHt,Other_NH,Kitchen_AbvGr,Mas_Vnr_Area,has_Second_Flr,Total_Bath,Crawfor,Overall_Cond,NWAmes,Somerst,Second_Flr_SF,Fireplaces,Garage_Cars,has_Garage,First_Flr_SF,has_Mas_Vnr,OldTown,Porch_Area,CulDSac,CollgCr,has_Porch,ratio_building_lot,Gr_Liv_Area,log_Lot_Area,log_Gr_Liv_Area,Edwards,m_Mas_Vnr_Area,Lot_Area,Garage_Area,m_Lot_Area,m_Second_Flr_SF,Full_Bath,Half_Bath,TotRms_AbvGr,BrkSide,Gilbert,Mitchel,NAmes,Sawyer,SawyerW
0,OLS Model (y),sklearn.linear_model._base,LinearRegression,0.83,0.86,0.03,True,-33515.84,18.73,18487.49,53860.77,16273.76,-28231.89,34.98,-25817.23,8731.39,20981.38,5013.05,-17245.46,21249.72,60.46,6194.89,17904.12,-20027.12,39.72,-6189.15,-10945.43,35.40,12113.65,9594.26,-7562.02,-50705.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OLS Model (log y),sklearn.linear_model._base,LinearRegression,0.87,0.86,0.01,True,5.46,0.00,0.10,0.14,NaN,-0.14,NaN,-0.22,0.06,0.07,0.05,-0.09,0.09,0.00,0.03,0.07,NaN,0.00,NaN,-0.14,0.00,0.05,0.05,NaN,NaN,-0.00,0.07,0.72,-0.04,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unscaled Ridge Model,sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
3,Unscaled Ridge Model (x_step_log_y),sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
4,Unscaled Lasso Model,sklearn.linear_model._coordinate_descent,Lasso,0.67,0.72,0.05,True,11.03,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00
5,Unscaled SGD Model,sklearn.linear_model._stochastic_gradient,SGDRegressor,-2337089720084032229485300861108224.00,-2055432209097418385988503174381568.00,281657510986613843496797686726656.00,True,-2143616213.39,5217070778249.79,280780621586.68,-33904257197.00,63787222983.43,8943009434.26,-2425309715169.90,12597414723.79,102565225240.90,27454549524.94,-235719225320.23,-5177465123.45,38960014907.97,492713685312.05,-112349891476.34,-141562840230.65,18761105319.66,-1794702045621.53,38725750303.76,-2647449698.61,-10262209880041.12,139722733918.26,-22464916099.23,-127373933158.57,15940147040.87,-3886462030840.47,-69732303105.68,2713321383.50,-83349027203.91,33442057384.00,1781867647044.51,103314447141.95,32291116572.22,-38033830911.13,16829317109.46,171471816262.88,-72542871880.37,-38036805989.86,9724481357.33,-56014254880.91,23565649191.59,-13278653305.02,89000523588.25


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Part V: Standardization</h2><br>
In this section, we will learn how to <strong>standardize</strong> the X-features. In other words, we are going to put them into a form where each feature's variance is measured on the same scale. Some algorithms base their calculations on distance, which requires standardization so that they work properly. Others have penalty terms that assume all features have a mean of zero and a standard deviation of one,  which is the result of standardization.
<br><br>
In general, distance- and penalty-based algorithms (like K-Nearest Neighbors, which we will see in the next script) perform much better after standardization. This is because distance-based algorithms use variance to compute similarity amongst observations: the closer two observations are in terms of their variance, the more similar the algorithm will think they are. Therefore, if the data is not standardized, features with less variance may take over the model. This can be a lot to conceptualize, so let's take it step by step, keeping in mind that our goal is to ensure that the variance in each feature is treated fairly by the algorithms we develop.
<br><br><strong>Standard Scaler</strong><br>
Technically speaking, this is our first unsupervised learning technique! Congrats on all that you've accomplished thus far! Notice how the process of data standardization is very similar to that of building models in scikit-learn:<br>

* Instantiate
* Fit
* <strike>Predict</strike> Transform
* <strike>Score</strike> Convert

<br>
<strong>a)</strong> Complete the code below to standardize all X-features.

In [13]:
# INSTANTIATING a StandardScaler() object
scaler = StandardScaler()


# FITTING and TRANSFORMING
x_scaled = scaler.fit_transform( housing[x_all] )


# converting scaled data into a DataFrame
x_scaled_df = pd.DataFrame(x_scaled)


# labeling columns
x_scaled_df.columns = x_data.columns


# checking the results
x_scaled_df.describe(include = 'number').round(decimals = 2)

,Lot_Area,Mas_Vnr_Area,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,Garage_Area,Porch_Area,m_Lot_Area,m_Mas_Vnr_Area,m_Second_Flr_SF,log_Lot_Area,log_Gr_Liv_Area,has_Second_Flr,has_Garage,has_Mas_Vnr,has_Porch,ratio_building_lot,Overall_Qual,Overall_Cond,Full_Bath,Half_Bath,Kitchen_AbvGr,TotRms_AbvGr,Fireplaces,Garage_Cars,Total_Bath,CulDSac,BrkSide,CollgCr,Crawfor,Edwards,Gilbert,Mitchel,NAmes,NWAmes,NridgHt,OldTown,Sawyer,SawyerW,Somerst,Other_NH
count,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00,2930.00
mean,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-1.12,-0.57,-2.38,-2.11,-0.78,-2.31,-2.20,-1.14,-0.18,-0.17,-0.28,-3.78,-4.46,-0.86,-4.19,-0.81,-2.25,-1.61,-3.61,-4.11,-2.83,-0.76,-4.88,-2.83,-0.93,-2.32,-2.73,-0.26,-0.20,-0.32,-0.19,-0.27,-0.24,-0.20,-0.42,-0.22,-0.25,-0.30,-0.23,-0.21,-0.26,-0.48
25%,-0.34,-0.57,-0.59,-0.72,-0.78,-0.74,-0.71,-0.84,-0.18,-0.17,-0.28,-0.35,-0.72,-0.86,0.24,-0.81,0.44,-0.55,-0.78,-0.51,-1.02,-0.76,-0.21,-0.92,-0.93,-1.01,-1.18,-0.26,-0.20,-0.32,-0.19,-0.27,-0.24,-0.20,-0.42,-0.22,-0.25,-0.30,-0.23,-0.21,-0.26,-0.48
50%,-0.09,-0.57,-0.14,-0.19,-0.78,-0.11,0.03,-0.11,-0.18,-0.17,-0.28,0.11,0.04,-0.86,0.24,-0.81,0.44,-0.22,-0.07,-0.51,0.78,-0.76,-0.21,-0.28,0.62,0.31,0.38,-0.26,-0.20,-0.32,-0.19,-0.27,-0.24,-0.20,-0.42,-0.22,-0.25,-0.30,-0.23,-0.21,-0.26,-0.48
75%,0.17,0.35,0.57,0.57,0.86,0.48,0.48,0.52,-0.18,-0.17,-0.28,0.51,0.63,1.16,0.24,1.24,0.44,0.14,0.64,0.39,0.78,1.23,-0.21,0.35,0.62,0.31,1.16,-0.26,-0.20,-0.32,-0.19,-0.27,-0.24,-0.20,-0.42,-0.22,-0.25,-0.30,-0.23,-0.21,-0.26,-0.48
max,26.07,8.39,11.47,10.04,4.04,8.19,4.72,7.77,5.62,5.79,3.54,6.28,4.24,1.16,0.24,1.24,0.44,8.12,2.77,3.09,4.40,3.22,9.14,5.44,5.25,4.25,5.05,3.91,5.11,3.16,5.24,3.76,4.09,4.97,2.37,4.62,4.08,3.36,4.29,4.74,3.89,2.10


<br><br>
<h3>Standardizing Candidate Models</h3><br>
Now that we've instantiated a standardized version of the x-data, we can standardize each candidate model with subsetting, as exemplified in the two codes below.

In [14]:
# x_all (not standardized)
housing[x_all].iloc[ : , 0:3 ].head(n=5)

,Lot_Area,Mas_Vnr_Area,Total_Bsmt_SF
0,31770,112,1080
1,11622,0,882
2,14267,108,1329
3,11160,0,2110
4,13830,0,928


In [15]:
# x_all (standardized)
x_scaled_df[x_all].iloc[ : , 0:3 ].head(n=5)

,Lot_Area,Mas_Vnr_Area,Total_Bsmt_SF
0,2.75,0.06,0.07
1,0.19,-0.57,-0.38
2,0.53,0.04,0.63
3,0.13,-0.57,2.40
4,0.47,-0.57,-0.28


<br><hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
<h2>Part VI: Team Challenge</h2>
<br>Below are the available candidate models.
<br><br>
<u><strong>Candidate Models (X-features)</strong></u>

* x_all
* x_original
* x_base
* x_step
* x_step_log_y


<br>
<u><strong>Response Variables (y)</strong></u>

* original_y
* log_y


<br>Your objectives in this challenge are to:

1. Determine which models perform better with standardized data.
2. Choose the best candidate model for each of the following model types:

* <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html">OLS Regression</a>
* <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html">Ridge Regression</a>
* <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html">Lasso Regression</a>
* <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html">SGD Regression</a>

<br>
<strong>a)</strong> Run each candidate model on each model type, recording its results in <em>results_df</em>.

In [16]:
# preparing x-data
x_data =  x_scaled_df [ x_all] # df can be housing or x_scaled_df

# preparing y-data
y_data = housing[ log_y ] # df can only be housing

<br>

In [17]:
# instantiating a ridge model
model = sklearn.linear_model.SGDRegressor(loss     = 'squared_error',
                                   penalty  = 'elasticnet',
                                   alpha    = 0.001,
                                   l1_ratio = 0.15,
                                   random_state = 702)


# analyzing results
results = lr_summary(x          = x_data,
                     y          = y_data,
                     model      = model,
                     model_name = 'SGD (Scaled) log_y',
                     results_df = results)


# checking results
results.head(n = 25)

,Model_Name,Model_Class,Model_Type,train_RSQ,test_RSQ,tt_gap,used_tts,Intercept,Total_Bsmt_SF,Overall_Qual,NridgHt,Other_NH,Kitchen_AbvGr,Mas_Vnr_Area,has_Second_Flr,Total_Bath,Crawfor,Overall_Cond,NWAmes,Somerst,Second_Flr_SF,Fireplaces,Garage_Cars,has_Garage,First_Flr_SF,has_Mas_Vnr,OldTown,Porch_Area,CulDSac,CollgCr,has_Porch,ratio_building_lot,Gr_Liv_Area,log_Lot_Area,log_Gr_Liv_Area,Edwards,m_Mas_Vnr_Area,Lot_Area,Garage_Area,m_Lot_Area,m_Second_Flr_SF,Full_Bath,Half_Bath,TotRms_AbvGr,BrkSide,Gilbert,Mitchel,NAmes,Sawyer,SawyerW
0,OLS Model (y),sklearn.linear_model._base,LinearRegression,0.83,0.86,0.03,True,-33515.84,18.73,18487.49,53860.77,16273.76,-28231.89,34.98,-25817.23,8731.39,20981.38,5013.05,-17245.46,21249.72,60.46,6194.89,17904.12,-20027.12,39.72,-6189.15,-10945.43,35.40,12113.65,9594.26,-7562.02,-50705.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OLS Model (log y),sklearn.linear_model._base,LinearRegression,0.87,0.86,0.01,True,5.46,0.00,0.10,0.14,NaN,-0.14,NaN,-0.22,0.06,0.07,0.05,-0.09,0.09,0.00,0.03,0.07,NaN,0.00,NaN,-0.14,0.00,0.05,0.05,NaN,NaN,-0.00,0.07,0.72,-0.04,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unscaled Ridge Model,sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
3,Unscaled Ridge Model (x_step_log_y),sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
4,Unscaled Lasso Model,sklearn.linear_model._coordinate_descent,Lasso,0.67,0.72,0.05,True,11.03,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00
5,Unscaled SGD Model,sklearn.linear_model._stochastic_gradient,SGDRegressor,-2337089720084032229485300861108224.00,-2055432209097418385988503174381568.00,281657510986613843496797686726656.00,True,-2143616213.39,5217070778249.79,280780621586.68,-33904257197.00,63787222983.43,8943009434.26,-2425309715169.90,12597414723.79,102565225240.90,27454549524.94,-235719225320.23,-5177465123.45,38960014907.97,492713685312.05,-112349891476.34,-141562840230.65,18761105319.66,-1794702045621.53,38725750303.76,-2647449698.61,-10262209880041.12,139722733918.26,-22464916099.23,-127373933158.57,15940147040.87,-3886462030840.47,-69732303105.68,2713321383.50,-83349027203.91,33442057384.00,1781867647044.51,103314447141.95,32291116572.22,-38033830911.13,16829317109.46,171471816262.88,-72542871880.37,-38036805989.86,9724481357.33,-56014254880.91,23565649191.59,-13278653305.02,89000523588.25
6,SGD (Scaled) log_y,sklearn.linear_model._stochastic_gradient,SGDRegressor,0.87,0.86,0.00,True,12.02,0.05,0.14,0.03,0.00,-0.02,0.01,-0.06,0.02,0.01,0.06,-0.02,0.02,0.01,0.02,0.05,0.00,-0.02,0.00,-0.04,0.02,0.01,0.01,-0.01,0.02,-0.05,0.08,0.18,-0.02,-0.01,-0.01,-0.00,0.00,0.00,0.02,0.01,-0.01,-0.01,0.00,0.00,-0.01,0.00,0.00


In [19]:
# instantiating a ridge model
model = sklearn.linear_model.SGDRegressor(loss     = 'squared_error',
                                   penalty  = 'elasticnet',
                                   alpha    = 0.1,
                                   l1_ratio = 0.20,
                                   random_state = 650)


# analyzing results
results = lr_summary(x          = x_data,
                     y          = y_data,
                     model      = model,
                     model_name = 'SGD (Scaled) log_y',
                     results_df = results)


# checking results
results.head(n = 25)

,Model_Name,Model_Class,Model_Type,train_RSQ,test_RSQ,tt_gap,used_tts,Intercept,Total_Bsmt_SF,Overall_Qual,NridgHt,Other_NH,Kitchen_AbvGr,Mas_Vnr_Area,has_Second_Flr,Total_Bath,Crawfor,Overall_Cond,NWAmes,Somerst,Second_Flr_SF,Fireplaces,Garage_Cars,has_Garage,First_Flr_SF,has_Mas_Vnr,OldTown,Porch_Area,CulDSac,CollgCr,has_Porch,ratio_building_lot,Gr_Liv_Area,log_Lot_Area,log_Gr_Liv_Area,Edwards,m_Mas_Vnr_Area,Lot_Area,Garage_Area,m_Lot_Area,m_Second_Flr_SF,Full_Bath,Half_Bath,TotRms_AbvGr,BrkSide,Gilbert,Mitchel,NAmes,Sawyer,SawyerW
0,OLS Model (y),sklearn.linear_model._base,LinearRegression,0.83,0.86,0.03,True,-33515.84,18.73,18487.49,53860.77,16273.76,-28231.89,34.98,-25817.23,8731.39,20981.38,5013.05,-17245.46,21249.72,60.46,6194.89,17904.12,-20027.12,39.72,-6189.15,-10945.43,35.40,12113.65,9594.26,-7562.02,-50705.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OLS Model (log y),sklearn.linear_model._base,LinearRegression,0.87,0.86,0.01,True,5.46,0.00,0.10,0.14,NaN,-0.14,NaN,-0.22,0.06,0.07,0.05,-0.09,0.09,0.00,0.03,0.07,NaN,0.00,NaN,-0.14,0.00,0.05,0.05,NaN,NaN,-0.00,0.07,0.72,-0.04,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Unscaled Ridge Model,sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
3,Unscaled Ridge Model (x_step_log_y),sklearn.linear_model._ridge,Ridge,0.88,0.86,0.01,True,5.74,0.00,0.09,0.14,0.01,-0.13,0.00,-0.21,0.03,0.06,0.05,-0.10,0.09,0.00,0.03,0.07,-0.01,0.00,-0.00,-0.15,0.00,0.04,0.05,-0.01,-0.06,-0.00,0.06,0.70,-0.05,-0.05,-0.00,-0.00,0.02,-0.00,0.02,0.03,-0.01,-0.03,0.01,0.02,-0.03,-0.01,-0.01
4,Unscaled Lasso Model,sklearn.linear_model._coordinate_descent,Lasso,0.67,0.72,0.05,True,11.03,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00
5,Unscaled SGD Model,sklearn.linear_model._stochastic_gradient,SGDRegressor,-2337089720084032229485300861108224.00,-2055432209097418385988503174381568.00,281657510986613843496797686726656.00,True,-2143616213.39,5217070778249.79,280780621586.68,-33904257197.00,63787222983.43,8943009434.26,-2425309715169.90,12597414723.79,102565225240.90,27454549524.94,-235719225320.23,-5177465123.45,38960014907.97,492713685312.05,-112349891476.34,-141562840230.65,18761105319.66,-1794702045621.53,38725750303.76,-2647449698.61,-10262209880041.12,139722733918.26,-22464916099.23,-127373933158.57,15940147040.87,-3886462030840.47,-69732303105.68,2713321383.50,-83349027203.91,33442057384.00,1781867647044.51,103314447141.95,32291116572.22,-38033830911.13,16829317109.46,171471816262.88,-72542871880.37,-38036805989.86,9724481357.33,-56014254880.91,23565649191.59,-13278653305.02,89000523588.25
6,SGD (Scaled) log_y,sklearn.linear_model._stochastic_gradient,SGDRegressor,0.87,0.86,0.00,True,12.02,0.05,0.14,0.03,0.00,-0.02,0.01,-0.06,0.02,0.01,0.06,-0.02,0.02,0.01,0.02,0.05,0.00,-0.02,0.00,-0.04,0.02,0.01,0.01,-0.01,0.02,-0.05,0.08,0.18,-0.02,-0.01,-0.01,-0.00,0.00,0.00,0.02,0.01,-0.01,-0.01,0.00,0.00,-0.01,0.00,0.00
7,SGD (Scaled) log_y,sklearn.linear_model._stochastic_gradient,SGDRegressor,0.86,0.85,0.01,True,12.02,0.04,0.14,0.03,0.00,-0.02,0.00,-0.04,0.03,0.01,0.06,-0.02,0.02,0.03,0.02,0.06,0.00,0.00,0.00,-0.04,0.01,0.01,0.01,0.00,0.00,0.03,0.04,0.04,-0.02,-0.02,-0.01,-0.00,0.00,0.00,0.01,0.01,0.00,-0.01,0.00,0.00,-0.01,0.00,0.00
8,SGD (Scaled) log_y,sklearn.linear_model._stochastic_gradient,SGDRegressor,0.83,0.84,0.01,True,12.02,0.05,0.15,0.01,0.00,-0.01,0.00,0.00,0.03,0.00,0.02,0.00,0.00,0.00,0.02,0.06,0.00,0.01,0.00,-0.02,0.01,0.00,0.00,0.00,0.00,0.03,0.03,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


<br>
<h3>Analysis</h3>

Use this markdown cell to write your analysis. Good luck!





<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

~~~


 █████╗ ██╗     ██╗ ██████╗ ███╗   ██╗██╗███╗   ██╗ ██████╗
██╔══██╗██║     ██║██╔════╝ ████╗  ██║██║████╗  ██║██╔════╝
███████║██║     ██║██║  ███╗██╔██╗ ██║██║██╔██╗ ██║██║  ███╗
██╔══██║██║     ██║██║   ██║██║╚██╗██║██║██║╚██╗██║██║   ██║
██║  ██║███████╗██║╚██████╔╝██║ ╚████║██║██║ ╚████║╚██████╔╝
╚═╝  ╚═╝╚══════╝╚═╝ ╚═════╝ ╚═╝  ╚═══╝╚═╝╚═╝  ╚═══╝ ╚═════╝
                                                            
██╗    ██╗██╗████████╗██╗  ██╗                              
██║    ██║██║╚══██╔══╝██║  ██║                              
██║ █╗ ██║██║   ██║   ███████║                              
██║███╗██║██║   ██║   ██╔══██║                              
╚███╔███╔╝██║   ██║   ██║  ██║                              
 ╚══╝╚══╝ ╚═╝   ╚═╝   ╚═╝  ╚═╝                              
                                                            
███████╗██╗   ██╗ ██████╗ ██████╗███████╗███████╗███████╗██╗
██╔════╝██║   ██║██╔════╝██╔════╝██╔════╝██╔════╝██╔════╝██║
███████╗██║   ██║██║     ██║     █████╗  ███████╗███████╗██║
╚════██║██║   ██║██║     ██║     ██╔══╝  ╚════██║╚════██║╚═╝
███████║╚██████╔╝╚██████╗╚██████╗███████╗███████║███████║██╗
╚══════╝ ╚═════╝  ╚═════╝ ╚═════╝╚══════╝╚══════╝╚══════╝╚═╝
                                                            


~~~

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br>
<h2>Bonus: Storing Model Results</h2><br>
The code below will store the model results from above as a new Excel file.

In [ ]:
# saving results in Excel
results_df.to_excel(excel_writer = "./model_results/model_results_2.xlsx",
                    index = False)

<br>